In [11]:
from pg_connect.connections import reading


ModuleNotFoundError: No module named 'pg_connect'

In [ ]:
print(df(head))


In [ ]:
# Ensure numeric columns are in the correct format
df['Total DL (Bytes)'] = pd.to_numeric(df['Total DL (Bytes)'], errors='coerce')
df['Total UL (Bytes)'] = pd.to_numeric(df['Total UL (Bytes)'], errors='coerce')


In [6]:
# Task 2.1: Aggregating Engagement Metrics per User
engagement_metrics = df.groupby('MSISDN/Number').agg(
    session_frequency=('Bearer Id', 'count'),
    session_duration=('Dur. (ms)', 'sum'),
    total_traffic=('Total DL (Bytes)', 'sum') + df.groupby('MSISDN/Number')['Total UL (Bytes)'].sum()
).reset_index()

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U16'), dtype('float64')) -> None

In [4]:
# Task 2.1: Aggregating Engagement Metrics per User
engagement_metrics = df.groupby('MSISDN/Number').agg(
    session_frequency=('Bearer Id', 'count'),
    session_duration=('Dur. (ms)', 'sum'),
    total_traffic=('Total DL (Bytes)', 'sum') + df['Total UL (Bytes)']
).reset_index()


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U16'), dtype('float64')) -> None

In [ ]:
# Normalize engagement metrics
scaler = StandardScaler()
engagement_metrics_scaled = scaler.fit_transform(engagement_metrics[['session_frequency', 'session_duration', 'total_traffic']])

In [ ]:
# Perform k-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
engagement_metrics['cluster'] = kmeans.fit_predict(engagement_metrics_scaled)

In [ ]:
# Compute cluster metrics
cluster_summary = engagement_metrics.groupby('cluster').agg(
    min_freq=('session_frequency', 'min'),
    max_freq=('session_frequency', 'max'),
    avg_freq=('session_frequency', 'mean'),
    min_duration=('session_duration', 'min'),
    max_duration=('session_duration', 'max'),
    avg_duration=('session_duration', 'mean'),
    min_traffic=('total_traffic', 'min'),
    max_traffic=('total_traffic', 'max'),
    avg_traffic=('total_traffic', 'mean')
).reset_index()

print("\nCluster Summary:")
print(cluster_summary)


In [ ]:
# Plot top 3 most used applications
top_apps = ['Social Media DL (Bytes)', 'YouTube DL (Bytes)', 'Netflix DL (Bytes)']
plt.figure(figsize=(12, 8))
for app in top_apps:
    plt.hist(df[app], bins=50, alpha=0.5, label=app)

plt.legend(loc='upper right')
plt.title('Top 3 Most Used Applications')
plt.xlabel('Data Volume (Bytes)')
plt.ylabel('Frequency')
plt.show()


In [1]:
# Optimal value of k using the elbow method
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(engagement_metrics_scaled)
    inertia.append(kmeans.inertia_)
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.show()